## Constructivo

Primero, vamos a intentar resolver el TOP a través de un algoritmo constructivo.
Recordemos que los parámetros son: $n$, $m$ y $T_{max}$

In [170]:
import pandas as pd
import os
from openpyxl import load_workbook
pd.set_option('display.max_rows', None)

# Definir la carpeta que contiene los archivos Excel
carpeta = "Files"

# Diccionario para almacenar los resultados por algoritmo, parámetros y hoja
resultados_completos = []

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".xlsx"):
        ruta_archivo = os.path.join(carpeta, archivo)
        
        # Extraer información sobre el algoritmo y los parámetros del nombre del archivo
        nombre_archivo = os.path.splitext(archivo)[0]
        partes_nombre = nombre_archivo.split("_")
        algoritmo = partes_nombre[2]
        parametros = tuple(partes_nombre[4:])
        if len(parametros) == 3: 
            parametros = parametros[1]+"_"+parametros[2]	# Para el caso de GRASP_RUIDO
            algoritmo = "GRASP_RUIDO"
        if algoritmo == "GRASP":
            parametros = parametros[0]
        elif algoritmo == "CONSTRUCTIVO":
            parametros = ()
        # Leer el valor obtenido al final de la primera columna en cada hoja del archivo Excel
        wb = load_workbook(filename=ruta_archivo, read_only=True)
        
        for hoja in wb.sheetnames:
            if hoja == 'Sheet': continue
            ws = wb[hoja]
            ultimo_valor = ws.cell(row=ws.max_row, column=1).value
            tiempo = ws.cell(row=ws.max_row, column=2).value
            
            # Actualizar el diccionario de resultados completos
            resultados_completos.append((algoritmo, parametros, hoja,ultimo_valor,tiempo))

df_resultados = pd.DataFrame(resultados_completos, columns=["Algoritmo", "Parámetros", "TOP", "Resultado","Tiempo"])

In [171]:
def obtener_cota(x):
    if x["TOP"] == "TOP1":
        return 285
    elif x["TOP"] == "TOP5":
        return 275
    elif x["TOP"] == "TOP9":
        return 1306
    elif x["TOP"] == "TOP13":
        return 1224
    elif x["TOP"] == "TOP17":
        return 1379
    elif x["TOP"] == "TOP21":
        return 1379

def mejor_por_top(x):
    top1 = x[x["TOP"] == "TOP1"]["Resultado"]
df_resultados["Cota"] = df_resultados.apply(obtener_cota, axis=1)
df_resultados["Gap"] = (df_resultados["Cota"] - df_resultados["Resultado"]) / df_resultados["Cota"] * 100
df_resultados = df_resultados.drop(df_resultados[df_resultados['Algoritmo'] == 'CONSTRUCTIVO'].index)
df_resultados.reset_index(drop=True, inplace=True)

In [172]:
grasp = df_resultados[df_resultados["Algoritmo"] == "GRASP"]
grasp_ruido = df_resultados[df_resultados["Algoritmo"] == "GRASP_RUIDO"]


Aquí se hace el análisis de parámetros para el Grasp. Se está comparando con las cotas superiores dadas. Con las estadísticas, sin hacer una prueba robusta, podríamos pensar que entre k2, k3 y k4 no hay mucha diferencia, mientras que k5 parece ser un poco peor. 

In [174]:
estadisticas_por_parametros_grasp = grasp.groupby("Parámetros")["Gap"].describe()

# Mostrar las estadísticas descriptivas
estadisticas_por_parametros_grasp


,count,mean,std,min,25%,50%,75%,max
Parámetros,,,,,,,,
k2,30.0,21.525461,11.663514,7.843137,9.029592,22.064679,32.272727,35.823060
k3,30.0,21.440973,11.944845,6.049005,8.823529,21.999160,32.727273,36.113125
k4,30.0,21.722725,11.835370,6.278714,9.647779,22.796840,31.381436,37.418419
k5,30.0,22.533192,12.288758,7.810107,8.977795,22.071677,34.545455,37.345903


In [182]:
estadisticas_por_parametros_grasp_ruido = grasp_ruido.groupby("Parámetros")["Gap"].describe().sort_values(by="mean")
estadisticas_por_parametros_grasp_ruido

,count,mean,std,min,25%,50%,75%,max
Parámetros,,,,,,,,
k2_r7,6.0,21.047519,12.296966,8.333333,10.689530,20.382915,30.578482,35.750544
k3_r8,6.0,21.296224,13.670418,7.843137,9.163174,20.143073,33.396401,36.330674
k2_r8,6.0,21.356732,11.851243,8.333333,11.856546,20.788647,30.505966,35.605511
k2_r6,6.0,21.429349,12.539584,8.333333,10.842669,20.145321,33.033819,34.880348
k3_r6,6.0,21.530921,12.771138,8.728943,10.055470,20.897421,31.923990,36.620740
k3_r10,6.0,21.597283,13.657363,6.661562,10.055470,21.513810,32.232184,37.853517
k2_r10,6.0,21.630083,12.967689,8.333333,10.708673,20.058618,33.124464,36.258158
k2_r9,6.0,21.651380,12.701700,8.823529,10.363437,21.006196,31.978377,36.620740
k3_r9,6.0,21.688345,12.432969,8.499234,10.494066,21.847131,31.960248,35.895577


In [180]:
# Agrupar los datos por algoritmo y hoja, y encontrar el índice del máximo valor en cada grupo
indices_maximos = df_resultados.groupby(["Algoritmo", "TOP"])["Resultado"].idxmax()

# Obtener las filas correspondientes a los índices máximos
mejores_combinaciones = df_resultados.loc[indices_maximos]

# Mostrar las mejores combinaciones
print(mejores_combinaciones)

       Algoritmo Parámetros    TOP  Resultado  Tiempo  Cota        Gap
114        GRASP         k5   TOP1        260   11926   285   8.771930
9          GRASP         k2  TOP13       1128   95180  1224   7.843137
40         GRASP         k3  TOP17        990  245687  1379  28.208847
5          GRASP         k2  TOP21        890  243354  1379  35.460479
19         GRASP         k2   TOP5        190   11582   275  30.909091
32         GRASP         k3   TOP9       1227  391003  1306   6.049005
168  GRASP_RUIDO      k3_r8   TOP1        260   12641   285   8.771930
171  GRASP_RUIDO      k3_r8  TOP13       1128   93283  1224   7.843137
130  GRASP_RUIDO      k2_r6  TOP17        986  252307  1379  28.498912
131  GRASP_RUIDO      k2_r6  TOP21        898  240046  1379  34.880348
133  GRASP_RUIDO      k2_r7   TOP5        190   12349   275  30.909091
152  GRASP_RUIDO     k3_r10   TOP9       1219  390638  1306   6.661562
